In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display_html 
from IPython import display
import sys
sys.path.append('C:/Users/User/Desktop/StockInfoHub')
from Shared_Modules.shared_variables import *
import warnings
import requests
import os
import datetime
warnings.filterwarnings('ignore')
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [67]:
df = pd.read_csv(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\history_data\^TWII.csv')
d = '2018-11-12'
print(df[df['Date'] == d])
df.head()

            Date         Open         High          Low        Close  \
4644  2018-11-12  9806.799805  9875.839844  9764.459961  9831.209961   

        Adj Close     Volume     ID         5MA         10MA  ...  \
4644  9831.209961  1553300.0  ^TWII  9867.96582  9830.920898  ...   

      RS 20MA 10MAX  RS 250EMA 10MAX  RS 50EMA 10MAX  RS 20EMA 10MAX  \
4644            0.0              0.0             0.0             0.0   

          ATR250      ATR50       ATR20      STD7     STD20     STD50  
4644  103.194694  128.95827  143.791201  0.449301  1.627079  5.237161  

[1 rows x 45 columns]


,Date,Open,High,Low,Close,Adj Close,Volume,ID,5MA,10MA,...,RS 20MA 10MAX,RS 250EMA 10MAX,RS 50EMA 10MAX,RS 20EMA 10MAX,ATR250,ATR50,ATR20,STD7,STD20,STD50
0,2000-01-04,8644.910156,8803.610352,8642.500000,8756.549805,8756.517578,0.0,^TWII,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-05,8690.599609,8867.679688,8668.019531,8849.870117,8849.837891,0.0,^TWII,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-06,8900.559570,9023.990234,8833.910156,8922.030273,8921.997070,0.0,^TWII,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-07,8853.429688,8940.769531,8739.429688,8849.870117,8849.837891,0.0,^TWII,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-10,8941.929688,9126.019531,8891.790039,9102.599609,9102.565430,0.0,^TWII,8896.151172,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
def get_tradingview_format():
    # ============上市股票df============
    url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
    response = requests.get(url, timeout=5)
    print(response.status_code)
    listed = pd.read_html(response.text)[0]
    listed.columns = listed.iloc[0,:]
    listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
    listed = listed.iloc[1:]

    # ============上櫃股票df============
    urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
    response = requests.get(urlTWO, timeout=5)
    print(response.status_code)
    listedTWO = pd.read_html(response.text)[0]
    listedTWO.columns = listedTWO.iloc[0,:]
    listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
    listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

    # ============上市股票代號+.TW============
    stock_1 = listed["有價證券代號"]
    stock_num = stock_1.apply(lambda x: str(x) + ".TW")
    stock_num.loc[len(stock_num)+1] = '0050.TW'
    stock_num.loc[len(stock_num)+1] = '^TWII'
    # print(stock_num)

    # ============上櫃股票代號+.TWO============
    stock_2 = listedTWO["有價證券代號"]
    stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
    # print(stock_num2)

    # ============concate全部股票代號============
    stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
    # print(stock_num)
    allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
    allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
    allstock_info.set_index('ID', inplace = True)
    # print(allstock_info)
    return allstock_info
folder_name = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/全個股條件篩選'
file_names = os.listdir(folder_name)
file_names = [file for file in file_names if file.endswith('.xlsx')]
file_names = [file.split('.')[0] for file in file_names]
file_names = sorted(file_names)
print(file_names)
industry_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx').astype(int).astype(str)
group_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\族群_複製.xlsx').astype(int).astype(str)
concept_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\概念股_複製.xlsx').astype(int).astype(str)
TAIEX_df = pd.read_csv(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\history_data\^TWII.csv')

all_industry_names = industry_df.columns.tolist()
all_industry_names = np.concatenate([all_industry_names, group_df.columns.tolist()])
all_industry_names = np.concatenate([all_industry_names, concept_df.columns.tolist()])
all_industry_names = np.unique(all_industry_names).tolist()
templates = ['T5', 'T5-2', 'T6', 'T11', 'T21', 'CF']

daily_template_industry = pd.DataFrame(np.zeros((len(file_names), len(templates)+1)), columns=['Date'] + templates)
daily_template_group = pd.DataFrame(np.zeros((len(file_names), len(templates)+1)), columns=['Date'] + templates)
daily_template_concept = pd.DataFrame(np.zeros((len(file_names), len(templates)+1)), columns=['Date'] + templates)
daily_template_df = pd.DataFrame(np.zeros((len(file_names), len(templates)+3)), columns=['Date'] + templates + ['TAIEX', 'TAIEX change %'])
for i, file in enumerate(file_names):
    file_path = folder_name + '/' + file + '.xlsx'
    df = pd.read_excel(file_path)
    ids = df['ID'].astype(str)
    if 'ID' in df.columns:
        df['ID'] = df['ID'].astype(str)
        df.set_index('ID', inplace = True)
        print(f'{bcolors.OK} {file} {bcolors.RESET} is loaded.')
    else:
        print(f'{bcolors.FAIL} {file} {bcolors.RESET} is not loaded.')
        continue
    try:
        a = allstock_info
    except NameError:
        allstock_info = get_tradingview_format()
    else:
        pass
    #看個股產業
    alist = []
    industry_category_df = []
    show = True
    df = df.sort_values(by='busness volume(億)', ascending=False)
    df['CF'] = [True if index_value < 50 else False for index_value, x in enumerate(df.index.values)]
    date = file.replace('選股', '')
    daily_template_industry.iloc[i, 0] = date
    daily_template_group.iloc[i, 0] = date
    daily_template_concept.iloc[i, 0] = date
    daily_template_df.iloc[i, 0] = date
    daily_template_df.loc[i, 'TAIEX'] = TAIEX_df[TAIEX_df['Date'] == date]['Close'].values[0]
    daily_template_df.loc[i, 'TAIEX change %'] = TAIEX_df[TAIEX_df['Date'] == date]['ROCP'].values[0]
    for template_index, template in enumerate(templates):
        to_watch = df[df[template] == True].index.values
        all_stock_industry = []
        all_stock_group = []
        all_stock_concept = []
        daily_template_df.loc[i, f'{template}'] = len(to_watch)
        for id in to_watch:
            stock_industry = []
            stock_group = []
            stock_concept = []
            for col in industry_df.columns.values:
                if id in industry_df[col].values:
                    stock_industry.append(col)
            for col in group_df.columns.values:
                if id in group_df[col].values:
                    stock_group.append(col)
            for col in concept_df.columns.values:
                if id in concept_df[col].values:
                    stock_concept.append(col)
            stock_industry = np.unique(stock_industry).tolist()
            stock_group = np.unique(stock_group).tolist()
            stock_concept = np.unique(stock_concept).tolist()
            all_stock_industry += stock_industry
            all_stock_group += stock_group
            all_stock_concept += stock_concept
        def count_industry(all_stock_industry):
            count_of_industry = pd.DataFrame(all_stock_industry, columns=['industry'])
            count_of_industry['count'] = 1
            count_of_industry = count_of_industry.groupby('industry').sum()
            count_of_industry = count_of_industry.sort_values(by='count', ascending=False)
            count_of_industry = count_of_industry.iloc[:3]
            text = ''
            for index in count_of_industry.index.values:
                text += f"{str(index)}({count_of_industry.loc[index, 'count'].astype(str)}) / "
            
            return text
        daily_template_industry.iloc[i, template_index+1] = count_industry(all_stock_industry)
        daily_template_group.iloc[i, template_index+1] = count_industry(all_stock_group)
        daily_template_concept.iloc[i, template_index+1] = count_industry(all_stock_concept)

    daily_template_industry.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股產業數量.xlsx', index=False)
    daily_template_group.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股族群數量.xlsx', index=False)
    daily_template_concept.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股概念股數量.xlsx', index=False)
    daily_template_df.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股數量.xlsx', index=False)

['2018-11-12選股', '2018-11-13選股', '2018-11-14選股', '2018-11-15選股', '2018-11-16選股', '2018-11-19選股', '2018-11-20選股', '2018-11-21選股', '2018-11-22選股', '2018-11-23選股', '2018-11-26選股', '2018-11-27選股', '2018-11-28選股', '2018-11-29選股', '2018-11-30選股', '2018-12-03選股', '2018-12-04選股', '2018-12-05選股', '2018-12-06選股', '2018-12-07選股', '2018-12-10選股', '2018-12-11選股', '2018-12-12選股', '2018-12-13選股', '2018-12-14選股', '2018-12-17選股', '2018-12-18選股', '2018-12-19選股', '2018-12-20選股', '2018-12-21選股', '2018-12-24選股', '2018-12-25選股', '2018-12-26選股', '2018-12-27選股', '2018-12-28選股', '2019-01-02選股', '2019-01-03選股', '2019-01-04選股', '2019-01-07選股', '2019-01-08選股', '2019-01-09選股', '2019-01-10選股', '2019-01-11選股', '2019-01-14選股', '2019-01-15選股', '2019-01-16選股', '2019-01-17選股', '2019-01-18選股', '2019-01-21選股', '2019-01-22選股', '2019-01-23選股', '2019-01-24選股', '2019-01-25選股', '2019-01-28選股', '2019-01-29選股', '2019-01-30選股', '2019-02-11選股', '2019-02-12選股', '2019-02-13選股', '2019-02-14選股', '2019-02-15選股', '2019-02-18選股', '2019-0

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display_html 
from IPython import display
import sys
sys.path.append('C:/Users/User/Desktop/StockInfoHub')
from Shared_Modules.shared_variables import *
import warnings
import requests
import os
import datetime
warnings.filterwarnings('ignore')
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR
def get_tradingview_format():
    # ============上市股票df============
    url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
    response = requests.get(url, timeout=5)
    print(response.status_code)
    listed = pd.read_html(response.text)[0]
    listed.columns = listed.iloc[0,:]
    listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
    listed = listed.iloc[1:]

    # ============上櫃股票df============
    urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
    response = requests.get(urlTWO, timeout=5)
    print(response.status_code)
    listedTWO = pd.read_html(response.text)[0]
    listedTWO.columns = listedTWO.iloc[0,:]
    listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
    listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

    # ============上市股票代號+.TW============
    stock_1 = listed["有價證券代號"]
    stock_num = stock_1.apply(lambda x: str(x) + ".TW")
    stock_num.loc[len(stock_num)+1] = '0050.TW'
    stock_num.loc[len(stock_num)+1] = '^TWII'
    # print(stock_num)

    # ============上櫃股票代號+.TWO============
    stock_2 = listedTWO["有價證券代號"]
    stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
    # print(stock_num2)

    # ============concate全部股票代號============
    stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
    # print(stock_num)
    allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
    allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
    allstock_info.set_index('ID', inplace = True)
    # print(allstock_info)
    return allstock_info
folder_name = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/全個股條件篩選'
n_day_ago = 2
file_names = str(datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=n_day_ago)) , '%Y-%m-%d' )).split(' ')[0]+'選股'
print(file_names)
industry_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx').astype(int).astype(str)
group_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\族群_複製.xlsx').astype(int).astype(str)
concept_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\概念股_複製.xlsx').astype(int).astype(str)
TAIEX_df = pd.read_csv(r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\history_data\^TWII.csv')

daily_template_industry_file = pd.read_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股產業數量.xlsx')
daily_template_group_file = pd.read_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股族群數量.xlsx')
daily_template_concept_file = pd.read_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股概念股數量.xlsx')
daily_template_df_file = pd.read_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股數量.xlsx')


all_industry_names = industry_df.columns.tolist()
all_industry_names = np.concatenate([all_industry_names, group_df.columns.tolist()])
all_industry_names = np.concatenate([all_industry_names, concept_df.columns.tolist()])
all_industry_names = np.unique(all_industry_names).tolist()
templates = ['T5', 'T5-2', 'T6', 'T11', 'T21', 'CF']

daily_template_industry = pd.DataFrame(np.zeros((len(file_names), len(templates)+1)), columns=['Date'] + templates)
daily_template_group = pd.DataFrame(np.zeros((len(file_names), len(templates)+1)), columns=['Date'] + templates)
daily_template_concept = pd.DataFrame(np.zeros((len(file_names), len(templates)+1)), columns=['Date'] + templates)
daily_template_df = pd.DataFrame(np.zeros((len(file_names), len(templates)+3)), columns=['Date'] + templates + ['TAIEX', 'TAIEX change %'])
for i, file in enumerate([file_names]):
    file_path = folder_name + '/' + file + '.xlsx'
    df = pd.read_excel(file_path)
    ids = df['ID'].astype(str)
    if 'ID' in df.columns:
        df['ID'] = df['ID'].astype(str)
        df.set_index('ID', inplace = True)
        print(f'{bcolors.OK} {file} {bcolors.RESET} is loaded.')
    else:
        print(f'{bcolors.FAIL} {file} {bcolors.RESET} is not loaded.')
        continue
    try:
        a = allstock_info
    except NameError:
        allstock_info = get_tradingview_format()
    else:
        pass
    #看個股產業
    alist = []
    industry_category_df = []
    show = True
    df = df.sort_values(by='busness volume(億)', ascending=False)
    df['CF'] = [True if index_value < 50 else False for index_value, x in enumerate(df.index.values)]
    date = file.replace('選股', '')
    daily_template_industry.iloc[i, 0] = date
    daily_template_group.iloc[i, 0] = date
    daily_template_concept.iloc[i, 0] = date
    daily_template_df.iloc[i, 0] = date
    daily_template_df.loc[i, 'TAIEX'] = TAIEX_df[TAIEX_df['Date'] == date]['Close'].values[0]
    daily_template_df.loc[i, 'TAIEX change %'] = TAIEX_df[TAIEX_df['Date'] == date]['ROCP'].values[0]
    for template_index, template in enumerate(templates):
        to_watch = df[df[template] == True].index.values
        all_stock_industry = []
        all_stock_group = []
        all_stock_concept = []
        daily_template_df.loc[i, f'{template}'] = len(to_watch)
        for id in to_watch:
            stock_industry = []
            stock_group = []
            stock_concept = []
            for col in industry_df.columns.values:
                if id in industry_df[col].values:
                    stock_industry.append(col)
            for col in group_df.columns.values:
                if id in group_df[col].values:
                    stock_group.append(col)
            for col in concept_df.columns.values:
                if id in concept_df[col].values:
                    stock_concept.append(col)
            stock_industry = np.unique(stock_industry).tolist()
            stock_group = np.unique(stock_group).tolist()
            stock_concept = np.unique(stock_concept).tolist()
            all_stock_industry += stock_industry
            all_stock_group += stock_group
            all_stock_concept += stock_concept
        def count_industry(all_stock_industry):
            count_of_industry = pd.DataFrame(all_stock_industry, columns=['industry'])
            count_of_industry['count'] = 1
            count_of_industry = count_of_industry.groupby('industry').sum()
            count_of_industry = count_of_industry.sort_values(by='count', ascending=False)
            count_of_industry = count_of_industry.iloc[:3]
            text = ''
            for index in count_of_industry.index.values:
                text += f"{str(index)}({count_of_industry.loc[index, 'count'].astype(str)}) / "
            
            return text
        daily_template_industry.iloc[i, template_index+1] = count_industry(all_stock_industry)
        daily_template_group.iloc[i, template_index+1] = count_industry(all_stock_group)
        daily_template_concept.iloc[i, template_index+1] = count_industry(all_stock_concept)

daily_template_concept_file = pd.concat([daily_template_concept_file, daily_template_concept], ignore_index=True)
daily_template_group_file = pd.concat([daily_template_group_file, daily_template_group], ignore_index=True)
daily_template_industry_file = pd.concat([daily_template_industry_file, daily_template_industry], ignore_index=True)
daily_template_df_file = pd.concat([daily_template_df_file, daily_template_df], ignore_index=True)    

daily_template_industry_file.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股產業數量.xlsx', index=False)
daily_template_group_file.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股族群數量.xlsx', index=False)
daily_template_concept_file.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股概念股數量.xlsx', index=False)
daily_template_df_file.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/每日策略選股數量.xlsx', index=False)

2024-05-03選股
 2024-05-03選股  is loaded.
200
200
